## Creates the doc2vec vector embeddings for a specific configuration

In [1]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import os
import time
from collections import namedtuple
import cPickle as pickle
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random
import gzip

from multiprocessing import Process, Queue

from multiprocessing.dummy import Pool as ThreadPool
import itertools

from sklearn.metrics import coverage_error
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.preprocessing import MultiLabelBinarizer

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import logging
from logging import info
from functools import partial

from thesis.utils.metrics import *
from thesis.utils.file import *

## Global variables used throughout the script

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
SVM_SEED = 1234
DOC2VEC_SEED = 1234

In [23]:
MIN_WORD_COUNT = 100
NUM_CORES = 16

In [5]:
GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL_NAME', 'DOC2VEC_MODEL', 
                                         'SVM_MODEL_NAME', 'NN_MODEL_NAME'])

In [6]:
VOCAB_MODEL = "vocab_model"
MODEL_PREFIX = "model"
VALIDATION_DICT = "validation_dict.pkl"
TEST_MATRIX = "test_matrix.pkl"
TEST_DICT = "test_dict.pkl"
METRICS = "metrics.pkl"
CLASSIFIER = "classifier.pkl"

In [7]:
root_location = "/mnt/virtual-machines/data/"
exports_location = root_location + "exported_data/"

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

preprocessed_location = root_location + "preprocessed_data/extended_pv_abs_desc_claims_full_chunks/"

training_preprocessed_files_prefix = preprocessed_location + "extended_pv_training_docs_data_preprocessed-"
validation_preprocessed_files_prefix = preprocessed_location + "extended_pv_validation_docs_data_preprocessed-"
test_preprocessed_files_prefix = preprocessed_location + "extended_pv_test_docs_data_preprocessed-"

## Load general data required for classification

In [8]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 19.2 s, sys: 1.26 s, total: 20.4 s
Wall time: 20.4 s


In [9]:
len(training_docs_list)

1286325

In [10]:
len(validation_docs_list)

321473

In [11]:
len(test_docs_list)

401877

# Utility functions for data loading

In [12]:
VALIDATION_MINI_BATCH_SIZE = 100000
def get_extended_docs_with_inference_data_only(doc2vec_model, file_to_write, preprocessed_files_prefix):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation or test documents
    """

    def infer_one_doc(doc_tuple):
        # doc2vec_model.random = np.random.RandomState(DOC2VEC_SEED)
        doc_id, doc_tokens = doc_tuple
        rep = doc2vec_model.infer_vector(doc_tokens)
        return (doc_id, rep)

    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)):
        info("===== Loading inference vectors")
        inference_documents_reps = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)))
        info("Loaded inference vectors matrix")
    else:
        inference_documents_reps = {}
        info("===== Getting vectors with inference")

        # Multi-threaded inference
#         inference_docs_iterator = ExtendedPVDocumentBatchGenerator(preprocessed_files_prefix, batch_size=None)
        inference_docs_iterator = BatchWrapper(preprocessed_files_prefix, batch_size=None)
        generator_func = inference_docs_iterator.__iter__()
        # map consumes the whole iterator on the spot, so we have to use itertools.islice to fake mini-batching
        mini_batch_size = VALIDATION_MINI_BATCH_SIZE
        batches_run = 1
        while True:
            pool = ThreadPool(NUM_CORES)
            threaded_reps_partial = pool.map(infer_one_doc, itertools.islice(generator_func, mini_batch_size))
            info("Finished: {} tags".format(batches_run * mini_batch_size))
            batches_run += 1
            if threaded_reps_partial:
                # threaded_reps.extend(threaded_reps_partial)
                inference_documents_reps.update(threaded_reps_partial)
            else:
                break
                
            pool.close()
            pool.terminate()

        pickle.dump(inference_documents_reps,
                    open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write), 'w'))

    return inference_documents_reps

In [13]:
class ExtendedPVDocumentBatchGenerator(Process):
    def __init__(self, filename_prefix, queue, batch_size=10000, start_file=0, offset=10000):
        super(ExtendedPVDocumentBatchGenerator, self).__init__()
        self.queue = queue
        self.offset = offset
        self.filename_prefix = filename_prefix
        self.files_loaded = start_file - offset

    def run(self):
        cur_file = None
        while True:
            try:
                if cur_file is None:
                    info("Loading new file for index: {}".format(str(self.files_loaded + self.offset)))
                    cur_file = gzip.open(self.filename_prefix + str(self.files_loaded + self.offset) + '.gz')
#                     cur_file = open(self.filename_prefix + str(self.files_loaded + self.offset))
                    self.files_loaded += self.offset
                for line in cur_file:
                    self.queue.put(line)
                cur_file.close()
                cur_file = None
            except IOError:
                self.queue.put(False, block=True, timeout=None)
                info("All files are loaded - last file: {}".format(str(self.files_loaded + self.offset)))
                return


class BatchWrapper(object):
    def __init__(self, training_preprocessed_files_prefix, buffer_size=10000, batch_size=10000, level=1, level_type=None):
        assert batch_size <= 10000 or batch_size is None
        self.level = level
        self.level_type = level_type[0] if level_type is not None else None
        self.batch_size = batch_size
        self.q = Queue(maxsize=buffer_size)
        self.p = ExtendedPVDocumentBatchGenerator(training_preprocessed_files_prefix, queue=self.q,
                                                  batch_size=batch_size, start_file=0, offset=10000)
        self.p.start()
        self.cur_data = []

    def is_correct_type(self, doc_id):
        parts = doc_id.split("_")
        len_parts = len(parts)
        if len_parts == self.level:
            if len_parts == 1:
                return True
            if len_parts == self.level and (parts[1][0] == self.level_type or self.level_type is None):
                return True
        return False

    def return_sentences(self, line):
        line_array = tuple(line.split(" "))
        doc_id = line_array[0]
        if not self.is_correct_type(doc_id):
            return False
        line_array = line_array[1:]
        len_line_array = len(line_array)
        # divide the document to batches according to the batch size
        sentences = []
        
        if self.batch_size is None:
            # dont use LabeledSentence for validation iterator
            sentences.append((doc_id, line_array))
        else:
            curr_batch_iter = 0
            while curr_batch_iter < len_line_array:
                sentences.append(LabeledSentence(words=line_array[curr_batch_iter: curr_batch_iter + self.batch_size], tags=[doc_id]))
                curr_batch_iter += self.batch_size
        return tuple(sentences)

    def __iter__(self):
        while True:
            item = self.q.get(block=True)
            if item is False:
                raise StopIteration()
            else:
                sentences = self.return_sentences(item)
                if not sentences:
                    None
                else:
                    for sentence in sentences:
                        yield sentence


# Doc2vec and SVM Parameters

In [14]:
DOC2VEC_SIZE = 200
DOC2VEC_WINDOW = 2
DOC2VEC_MAX_VOCAB_SIZE = None
DOC2VEC_SAMPLE = 1e-3
DOC2VEC_TYPE = 1
DOC2VEC_HIERARCHICAL_SAMPLE = 0
DOC2VEC_NEGATIVE_SAMPLE_SIZE = 10
DOC2VEC_CONCAT = 0
DOC2VEC_MEAN = 1
DOC2VEC_TRAIN_WORDS = 0
DOC2VEC_EPOCHS = 1 # we do our training manually one epoch at a time
DOC2VEC_MAX_EPOCHS = 8
REPORT_DELAY = 20 # report the progress every x seconds
REPORT_VOCAB_PROGRESS = 100000 # report vocab progress every x documents

## Create the Doc2vec model and create/load the vocab

In [24]:
models = [
    (3, 'description')
]
level, model_name = models[0]

In [25]:
info("creating/loading vocabulary for " + str(level) + ' ' + model_name + ' in ')
doc2vec_model_save_location = os.path.join(root_location,
                                           "parameter_search_doc2vec_models_" + str(level) + '_' + model_name,
                                           "full")
if not os.path.exists(doc2vec_model_save_location):
    os.makedirs(doc2vec_model_save_location)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL)):
    os.makedirs(os.path.join(doc2vec_model_save_location, VOCAB_MODEL))

placeholder_model_name = 'doc2vec_size_{}_w_{}_type_{}_concat_{}_mean_{}_trainwords_{}_hs_{}_neg_{}_vocabsize_{}_model_{}'.format(DOC2VEC_SIZE,
                                                                DOC2VEC_WINDOW,
                                                                'dm' if DOC2VEC_TYPE == 1 else 'pv-dbow',
                                                                DOC2VEC_CONCAT, DOC2VEC_MEAN,
                                                                DOC2VEC_TRAIN_WORDS,
                                                                DOC2VEC_HIERARCHICAL_SAMPLE,DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                                                                str(DOC2VEC_MAX_VOCAB_SIZE),
                                                                str(level) + '_' + model_name
                                                                )
GLOBAL_VARS.DOC2VEC_MODEL_NAME = placeholder_model_name
placeholder_model_name = os.path.join(placeholder_model_name, "epoch_{}")
info("FILE " + os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
doc2vec_model = Doc2Vec(size=DOC2VEC_SIZE, window=DOC2VEC_WINDOW, min_count=MIN_WORD_COUNT,
                max_vocab_size= DOC2VEC_MAX_VOCAB_SIZE,
                sample=DOC2VEC_SAMPLE, seed=DOC2VEC_SEED, workers=NUM_CORES,
                # doc2vec algorithm dm=1 => PV-DM, dm=2 => PV-DBOW, PV-DM dictates CBOW for words
                dm=DOC2VEC_TYPE,
                # hs=0 => negative sampling, hs=1 => hierarchical softmax
                hs=DOC2VEC_HIERARCHICAL_SAMPLE, negative=DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                dm_concat=DOC2VEC_CONCAT,
                # would train words with skip-gram on top of cbow, we don't need that for now
                dbow_words=DOC2VEC_TRAIN_WORDS,
                iter=DOC2VEC_EPOCHS)

GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model

if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX)):
    
    info("Creating vocab model")
    training_docs_iterator = BatchWrapper(training_preprocessed_files_prefix, batch_size=10000, level=level,
                                          level_type=model_name)
    doc2vec_model.build_vocab(sentences=training_docs_iterator, progress_per=REPORT_VOCAB_PROGRESS)
    doc2vec_model.save(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
else:
    info("Loading vocab model")
    doc2vec_model_vocab_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
    doc2vec_model.reset_from(doc2vec_model_vocab_model)

2017-04-09 19:07:40,351 : INFO : creating/loading vocabulary for 3 description in 
2017-04-09 19:07:40,353 : INFO : FILE /mnt/virtual-machines/data/parameter_search_doc2vec_models_3_description/full/vocab_model/model
2017-04-09 19:07:41,520 : INFO : Loading vocab model
2017-04-09 19:07:41,521 : INFO : loading Doc2Vec object from /mnt/virtual-machines/data/parameter_search_doc2vec_models_3_description/full/vocab_model/model
2017-04-09 19:10:18,403 : INFO : loading docvecs recursively from /mnt/virtual-machines/data/parameter_search_doc2vec_models_3_description/full/vocab_model/model.docvecs.* with mmap=None
2017-04-09 19:10:18,404 : INFO : loading doctag_syn0 from /mnt/virtual-machines/data/parameter_search_doc2vec_models_3_description/full/vocab_model/model.docvecs.doctag_syn0.npy with mmap=None
2017-04-09 19:10:29,564 : INFO : loading doctag_syn0_lockf from /mnt/virtual-machines/data/parameter_search_doc2vec_models_3_description/full/vocab_model/model.docvecs.doctag_syn0_lockf.npy wit

## Actual Training, validation and Metrics Loop

In [ ]:
doc2vec_model.min_alpha = 0.025
DOC2VEC_ALPHA_DECREASE = 0.001

In [19]:
doc2vec_model.workers = NUM_CORES

In [ ]:
%%time
# when resuming, resume from an epoch with a previously created doc2vec model to get the learning rate right
start_from = 1
for epoch in range(start_from, DOC2VEC_MAX_EPOCHS+1):
    GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)
    info("****************** Epoch {} --- Working on {} *******************".format(epoch, GLOBAL_VARS.MODEL_NAME))
    
    # if we have the model, just load it, otherwise train the previous model
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
        doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        doc2vec_model.workers = NUM_CORES
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
    else:
        # train the doc2vec model
        training_docs_iterator = BatchWrapper(training_preprocessed_files_prefix, batch_size=10000, level=level,
                                          level_type=model_name)
        %time doc2vec_model.train(sentences=training_docs_iterator, report_delay=REPORT_DELAY)
        doc2vec_model.alpha -= DOC2VEC_ALPHA_DECREASE  # decrease the learning rate
        doc2vec_model.min_alpha = doc2vec_model.alpha  # fix the learning rate, no decay
        ensure_disk_location_exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME))
        doc2vec_model.save(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
        
    # only do the inference for higher epochs, as inference usually takes as much time as the actual training
    if epoch > 7:
        # Validation Embeddings
        info('Getting Validation Embeddings')
        Xv = get_extended_docs_with_inference_data_only(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix)

2017-04-09 19:16:53,277 : INFO : ****************** Epoch 1 --- Working on doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_3_description/epoch_1 *******************
2017-04-09 19:16:54,699 : INFO : Loading new file for index: 0
2017-04-09 19:16:54,709 : INFO : training model with 16 workers on 434313 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=10 window=2
2017-04-09 19:16:54,711 : INFO : expecting 29590673 sentences, matching count from corpus used for vocabulary survey
2017-04-09 19:16:55,756 : INFO : PROGRESS: at 0.00% examples, 160047 words/s, in_qsize 31, out_qsize 0
2017-04-09 19:17:15,767 : INFO : PROGRESS: at 0.12% examples, 422908 words/s, in_qsize 30, out_qsize 0
2017-04-09 19:17:35,768 : INFO : PROGRESS: at 0.23% examples, 428560 words/s, in_qsize 31, out_qsize 0
2017-04-09 19:17:55,781 : INFO : PROGRESS: at 0.35% examples, 432663 words/s, in_qsize 31, out_qsize 0
2017-04-09 19:18:15,785 : INFO : PROGRESS: at 0

2017-04-09 19:39:36,639 : INFO : PROGRESS: at 8.18% examples, 456300 words/s, in_qsize 29, out_qsize 0
2017-04-09 19:39:56,651 : INFO : PROGRESS: at 8.30% examples, 456885 words/s, in_qsize 17, out_qsize 0
2017-04-09 19:40:16,677 : INFO : PROGRESS: at 8.43% examples, 457416 words/s, in_qsize 32, out_qsize 0
2017-04-09 19:40:31,293 : INFO : Loading new file for index: 110000
2017-04-09 19:40:36,689 : INFO : PROGRESS: at 8.56% examples, 457758 words/s, in_qsize 31, out_qsize 0
2017-04-09 19:40:56,693 : INFO : PROGRESS: at 8.69% examples, 458072 words/s, in_qsize 31, out_qsize 0
2017-04-09 19:41:16,698 : INFO : PROGRESS: at 8.81% examples, 458602 words/s, in_qsize 28, out_qsize 0
2017-04-09 19:41:36,698 : INFO : PROGRESS: at 8.94% examples, 458837 words/s, in_qsize 23, out_qsize 0
2017-04-09 19:41:56,711 : INFO : PROGRESS: at 9.07% examples, 459365 words/s, in_qsize 31, out_qsize 0
2017-04-09 19:42:16,719 : INFO : PROGRESS: at 9.19% examples, 459840 words/s, in_qsize 28, out_qsize 0
2017-

2017-04-09 20:03:37,492 : INFO : PROGRESS: at 17.80% examples, 481280 words/s, in_qsize 29, out_qsize 0
2017-04-09 20:03:44,780 : INFO : Loading new file for index: 230000
2017-04-09 20:03:57,492 : INFO : PROGRESS: at 17.94% examples, 481507 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:04:17,537 : INFO : PROGRESS: at 18.08% examples, 481730 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:04:37,569 : INFO : PROGRESS: at 18.21% examples, 481999 words/s, in_qsize 32, out_qsize 0
2017-04-09 20:04:57,632 : INFO : PROGRESS: at 18.34% examples, 482264 words/s, in_qsize 32, out_qsize 0
2017-04-09 20:05:17,637 : INFO : PROGRESS: at 18.48% examples, 482520 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:05:37,639 : INFO : PROGRESS: at 18.62% examples, 482694 words/s, in_qsize 32, out_qsize 0
2017-04-09 20:05:39,864 : INFO : Loading new file for index: 240000
2017-04-09 20:05:57,650 : INFO : PROGRESS: at 18.74% examples, 482946 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:06:17,656 : INFO :

2017-04-09 20:27:18,313 : INFO : PROGRESS: at 27.43% examples, 492042 words/s, in_qsize 32, out_qsize 0
2017-04-09 20:27:38,317 : INFO : PROGRESS: at 27.56% examples, 492188 words/s, in_qsize 29, out_qsize 0
2017-04-09 20:27:58,318 : INFO : PROGRESS: at 27.69% examples, 492355 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:28:18,334 : INFO : PROGRESS: at 27.83% examples, 492472 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:28:37,728 : INFO : Loading new file for index: 360000
2017-04-09 20:28:38,340 : INFO : PROGRESS: at 27.96% examples, 492609 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:28:58,376 : INFO : PROGRESS: at 28.10% examples, 492661 words/s, in_qsize 32, out_qsize 0
2017-04-09 20:29:18,402 : INFO : PROGRESS: at 28.24% examples, 492756 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:29:38,412 : INFO : PROGRESS: at 28.36% examples, 492869 words/s, in_qsize 27, out_qsize 0
2017-04-09 20:29:58,417 : INFO : PROGRESS: at 28.50% examples, 492979 words/s, in_qsize 31, out_qsiz

2017-04-09 20:50:59,120 : INFO : PROGRESS: at 37.15% examples, 498070 words/s, in_qsize 32, out_qsize 0
2017-04-09 20:51:19,135 : INFO : PROGRESS: at 37.28% examples, 498171 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:51:19,698 : INFO : Loading new file for index: 480000
2017-04-09 20:51:39,140 : INFO : PROGRESS: at 37.42% examples, 498230 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:51:59,141 : INFO : PROGRESS: at 37.56% examples, 498281 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:52:19,156 : INFO : PROGRESS: at 37.69% examples, 498330 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:52:39,177 : INFO : PROGRESS: at 37.83% examples, 498394 words/s, in_qsize 29, out_qsize 0
2017-04-09 20:52:59,191 : INFO : PROGRESS: at 37.98% examples, 498425 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:53:11,951 : INFO : Loading new file for index: 490000
2017-04-09 20:53:19,202 : INFO : PROGRESS: at 38.12% examples, 498515 words/s, in_qsize 31, out_qsize 0
2017-04-09 20:53:39,206 : INFO :

...

2017-04-11 04:13:44,221 : INFO : PROGRESS: at 83.38% examples, 503081 words/s, in_qsize 32, out_qsize 0
2017-04-11 04:14:04,228 : INFO : PROGRESS: at 83.54% examples, 503181 words/s, in_qsize 31, out_qsize 0
2017-04-11 04:14:24,237 : INFO : PROGRESS: at 83.69% examples, 503282 words/s, in_qsize 31, out_qsize 0
2017-04-11 04:14:44,248 : INFO : PROGRESS: at 83.84% examples, 503383 words/s, in_qsize 30, out_qsize 0
2017-04-11 04:14:55,677 : INFO : Loading new file for index: 1080000
2017-04-11 04:15:04,249 : INFO : PROGRESS: at 84.00% examples, 503483 words/s, in_qsize 30, out_qsize 0
2017-04-11 04:15:24,250 : INFO : PROGRESS: at 84.15% examples, 503591 words/s, in_qsize 31, out_qsize 0
2017-04-11 04:15:44,259 : INFO : PROGRESS: at 84.31% examples, 503686 words/s, in_qsize 32, out_qsize 0
2017-04-11 04:16:04,270 : INFO : PROGRESS: at 84.46% examples, 503784 words/s, in_qsize 31, out_qsize 0
2017-04-11 04:16:24,277 : INFO : PROGRESS: at 84.61% examples, 503882 words/s, in_qsize 31, out_qsi

2017-04-11 04:37:25,006 : INFO : PROGRESS: at 93.74% examples, 507625 words/s, in_qsize 31, out_qsize 0
2017-04-11 04:37:45,011 : INFO : PROGRESS: at 93.88% examples, 507678 words/s, in_qsize 31, out_qsize 0
2017-04-11 04:38:05,027 : INFO : PROGRESS: at 94.03% examples, 507733 words/s, in_qsize 26, out_qsize 0
2017-04-11 04:38:06,132 : INFO : Loading new file for index: 1210000
2017-04-11 04:38:25,057 : INFO : PROGRESS: at 94.17% examples, 507798 words/s, in_qsize 30, out_qsize 1
2017-04-11 04:38:45,068 : INFO : PROGRESS: at 94.32% examples, 507866 words/s, in_qsize 28, out_qsize 0
2017-04-11 04:39:05,133 : INFO : PROGRESS: at 94.46% examples, 507919 words/s, in_qsize 32, out_qsize 0
2017-04-11 04:39:25,136 : INFO : PROGRESS: at 94.61% examples, 507970 words/s, in_qsize 32, out_qsize 0
2017-04-11 04:39:45,144 : INFO : PROGRESS: at 94.75% examples, 508023 words/s, in_qsize 31, out_qsize 0
2017-04-11 04:39:54,223 : INFO : Loading new file for index: 1220000
2017-04-11 04:40:05,146 : INFO

CPU times: user 18h 45min 5s, sys: 44min 33s, total: 19h 29min 38s
Wall time: 4h 7min 32s


2017-04-11 04:52:44,649 : INFO : storing np array 'doctag_syn0_lockf' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_3_description/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_3_description/epoch_8/model.docvecs.doctag_syn0_lockf.npy
2017-04-11 04:52:44,732 : INFO : not storing attribute syn0norm
2017-04-11 04:52:44,733 : INFO : storing np array 'syn0' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_3_description/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_3_description/epoch_8/model.wv.syn0.npy
2017-04-11 04:52:45,170 : INFO : storing np array 'syn1neg' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_3_description/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_3_description/epoch_8/model.syn1neg.npy
2017-04-11 04:52:45,379 : INFO : not storing attribute cum_table
2017-04-11 04:56:52,966 : INFO

NameError: global name 'BatchClass' is not defined

In [46]:
NUM_CORES = 32

In [47]:
Xv = get_extended_docs_with_inference_data_only(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix)

2017-04-09 00:47:30,381 : INFO : ===== Getting vectors with inference
2017-04-09 00:47:32,316 : INFO : Loading new file for index: 0
2017-04-09 00:48:09,470 : INFO : Finished: 100000 tags
2017-04-09 00:48:46,768 : INFO : Finished: 200000 tags
2017-04-09 00:49:24,072 : INFO : Finished: 300000 tags
2017-04-09 00:49:30,546 : INFO : Loading new file for index: 10000
2017-04-09 00:50:01,360 : INFO : Finished: 400000 tags
2017-04-09 00:50:39,518 : INFO : Finished: 500000 tags
2017-04-09 00:51:17,982 : INFO : Finished: 600000 tags
2017-04-09 00:51:31,647 : INFO : Loading new file for index: 20000
2017-04-09 00:51:56,195 : INFO : Finished: 700000 tags
2017-04-09 00:52:32,945 : INFO : Finished: 800000 tags
2017-04-09 00:53:10,328 : INFO : Finished: 900000 tags
2017-04-09 00:53:47,587 : INFO : Finished: 1000000 tags
2017-04-09 00:53:51,325 : INFO : Loading new file for index: 30000
2017-04-09 00:54:25,132 : INFO : Finished: 1100000 tags
2017-04-09 00:55:03,058 : INFO : Finished: 1200000 tags
201